# 推荐评论展示任务

# 任务描述 

本次推荐评论展示任务的目标是从真实的用户评论中，挖掘合适作为推荐理由的短句。点评软件展示的推荐理由具有长度限制，而真实用户评论语言通顺、信息完整。综合来说，两者都具有用户情感的正负向，但是展示推荐理由的内容相关性高于评论，需要较强的文本吸引力。一些真实的推荐理由如下图所示：

![推荐理由](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/textcls_pic/apppic.png)



# 数据集

本次推荐评论展示任务所采用的数据集是点评软件中，用户中文评论的集合。


## 数据样例
本次任务要求将这些评论分为两类，即“展示”和“不展示”，分别以数字1和0作为标注，如下图所示：


![数据样例](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/textcls_pic/dataexp.png)

## 文档说明 


数据集文件分为训练集和测试集部分，对应文件如下：

- 带标签的训练数据：`train_shuffle.txt` 
- 不带标签的测试数据：`test_handout.txt`

注意，`test_handout.txt`文件的行索引从0开始，对应于ID一列，评论内容为“展示”的预测概率应于Prediction一列。

需要注意的是，由于数据在标注时存在主观偏好，标记为“不展示”（0）的评论不一定是真正的负面评论，反之亦然。但是这种情况的存在，不会对任务造成很大的歧义，通过基准算法我们可以在测试集上实现很高的性能。

# 任务拆解

推荐评论展示任务需要对数据集中的评论文本文件进行处理，传统的学习方式流程为：

![学习流程](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/textcls_pic/method.png)

对于以上的任务，近年来兴起的深度学习算法都可以实现：

![学习流程](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/textcls_pic/deepmethod.png)

# 评估说明

## 评价指标

本次任务采用 [AUC（Area Under Curve)](https://baike.baidu.com/item/AUC/19282953) 作为模型的评价标准。

## 在线评估

评估函数首先会验证选手提交的预测结果文件是否符合要求，主要验证了以下要求:

1. 提交的预测文件是否存在重复ID
2. 提交的预测文件ID是否与测试集文件ID不匹配
3. 提交的预测文件Prediction列是否存在整数（auc测评要求选手提供的是概率值，而非分类结果0或1）

通过验证后的文件会用以AUC为测评指标的函数进行计算评估。


## 文件格式

由于测评脚本已经统一，为保证脚本的顺利运行，在进行测评时，要求选手提交的`预测文件`拥有规范的字段名和字段格式，预测文件具体要求如下：

| NO | 字段名称 | 数据类型 | 字段描述 |
| -------- | -------- | -------- | -------- |
| 1    | ID     | int    | ID序列     |
| 2    | Prediction   | float     | 预测结果（概率值）   |

正确格式的提交文件样例: `submission_random.csv`。

## 基准算法

本次任务采用不同的基准算法，获得模型的AUC如下：
- 随机基准算法AUC：0.51209
- 弱基准算法AUC：0.85107
- 强基准算法AUC：0.94452
在评估时，以弱基准算法的AUC作为达标线。


## 终审评估

本次任务的终审评估将挑选在评分指标位于前10名的同学进行项目报告撰写，以描述模型、算法及实验等相关内容和结果，报告排版要求届时发布。

除此以外，为保证竞赛的公平性，进入终审评估的同学需要提交项目代码，由助教进行模型的有效性验证。

如发现实验结果有较大差异，或者模型无法复现等问题，组委会将取消营员本次14天陪你挑战《动手学深度学习》的结营资格，并且进行公示。

In [1]:
pip install torchtext

You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import collections
import os
import random
import time
from tqdm import tqdm
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
import torch.nn.functional as F
import numpy as np
from torch import nn, optim
import sys
import math
import re
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def read_comments_trian():
    with open('/home/kesci/work/Comments9120/train_shuffle.txt') as f:
        tags = [re.sub('[^0-1]+', ' ', line.strip()) for line in f]
    with open('/home/kesci/work/Comments9120/train_shuffle.txt') as f:
        lines = [line.strip().replace('0', '').replace('1', '').replace('\t', '') for line in f]
        line_nums=range(0,len(lines))
    return tags,lines,line_nums

#train_shuffle
tags,lines,line_nums = read_comments_trian()
train_data=[]
for i in range(0,len(tags)):
    train_data.append([lines[i],int(tags[i])])

In [4]:
def get_tokenized_imdb(data):
    '''
    @params:
        data: 数据的列表，列表中的每个元素为 [文本字符串，0/1标签] 二元组
    @return: 切分词后的文本的列表，列表中的每个元素为切分后的词序列
    '''
    back=[]
    for line in data:
        tempt=[]
        for word in line[0]:
            tempt.append(word)
        back.append(tempt)
    return back
def get_vocab_imdb(data):
    '''
    @params:
        data: 同上
    @return: 数据集上的词典，Vocab 的实例（freqs, stoi, itos）
    '''
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return Vocab.Vocab(counter, min_freq=0)

vocab = get_vocab_imdb(train_data)

In [5]:
def preprocess_imdb(data, vocab):
    '''
    @params:
        data: 同上，原始的读入数据
        vocab: 训练集上生成的词典
    @return:
        features: 单词下标序列，形状为 (n, max_l) 的整数张量
        labels: 情感标签，形状为 (n,) 的0/1整数张量
    '''
    max_l = 500  # 将每条评论通过截断或者补0，使得长度变成500

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(data)
    features = torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    labels = torch.tensor([score for _, score in data])
    return features, labels
train_set = Data.TensorDataset(*preprocess_imdb(train_data, vocab))
batch_size = 64
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)

In [6]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers):
        '''
        @params:
            vocab: 在数据集上创建的词典，用于获取词典大小
            embed_size: 嵌入维度大小
            num_hiddens: 隐藏状态维度大小
            num_layers: 隐藏层个数
        '''
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        
        # encoder-decoder framework
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embed_size, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                bidirectional=True)
        self.decoder = nn.Linear(4*num_hiddens, 2) # 初始时间步和最终时间步的隐藏状态作为全连接层输入
        
    def forward(self, inputs):
        '''
        @params:
            inputs: 词语下标序列，形状为 (batch_size, seq_len) 的整数张量
        @return:
            outs: 对文本情感的预测，形状为 (batch_size, 2) 的张量
        '''
        # 因为LSTM需要将序列长度(seq_len)作为第一维，所以需要将输入转置
        embeddings = self.embedding(inputs.permute(1, 0)) # (seq_len, batch_size, d)
        # rnn.LSTM 返回输出、隐藏状态和记忆单元，格式如 outputs, (h, c)
        outputs, _ = self.encoder(embeddings) # (seq_len, batch_size, 2*h)
        encoding = torch.cat((outputs[0], outputs[-1]), -1) # (batch_size, 4*h)
        outs = self.decoder(encoding) # (batch_size, 2)
        return outs

In [7]:
embed_size, num_hiddens, num_layers = 100, 100, 2
net = BiRNN(vocab, embed_size, num_hiddens, num_layers)
cache_dir = "/home/kesci/input/GloVe6B5429"
glove_vocab = Vocab.GloVe(name='6B', dim=100, cache=cache_dir)
def load_pretrained_embedding(words, pretrained_vocab):
    '''
    @params:
        words: 需要加载词向量的词语列表，以 itos (index to string) 的词典形式给出
        pretrained_vocab: 预训练词向量
    @return:
        embed: 加载到的词向量
    '''
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    if oov_count > 0:
        print("There are %d oov words." % oov_count)
    return embed
net.embedding.weight.data.copy_(load_pretrained_embedding(vocab.itos, glove_vocab))
net.embedding.weight.requires_grad = True # 词向量不匹配需要更新

There are 2475 oov words.


In [8]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        device = list(net.parameters())[0].device 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval()
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train()
            else:
                if('is_training' in net.__code__.co_varnames):
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

def train(train_iter,  net, loss, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y) 
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        #test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f,  time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, time.time() - start))
lr, num_epochs = 0.01, 5
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
train(train_iter, net, loss, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.3839, train acc 0.834,  time 67.8 sec
epoch 2, loss 0.1246, train acc 0.902,  time 67.6 sec
epoch 3, loss 0.0716, train acc 0.916,  time 67.7 sec
epoch 4, loss 0.0473, train acc 0.926,  time 67.6 sec
epoch 5, loss 0.0342, train acc 0.937,  time 67.7 sec


In [9]:
def predict_sentiment(net, vocab, sentence):
    '''
    @params：
        net: 训练好的模型
        vocab: 在该数据集上创建的词典，用于将给定的单词序转换为单词下标的序列，从而输入模型
        sentence: 需要分析情感的文本，以单词序列的形式给出
    @return: 预测的结果，positive 为正面情绪文本，negative 为负面情绪文本
    '''
    device = list(net.parameters())[0].device # 读取模型所在的环境
    sentence = torch.tensor([vocab.stoi[word] for word in sentence], device=device)
    label = net(sentence.view((1, -1)))
    tempt0=label[0][0].item()
    tempt1=label[0][1].item()
    result1=np.exp(tempt1)/(np.exp(tempt0)+np.exp(tempt1))
    return result1


In [10]:
def read_comments_test():
    with open('/home/kesci/work/Comments9120/test_handout.txt') as f:
        test_item= [line.strip().replace('0', '').replace('1', '') for line in f]
        test_nums=range(0,len(lines))
    return test_item,test_nums
test_item,test_nums=read_comments_test()

In [11]:
test_set=[]
for line in test_item:
    tempt=[]
    for word in line:
        tempt.append(word)
    test_set.append(tempt)
prob=[]
for test_text in test_set:
    tempt=predict_sentiment(net, vocab, test_text)
    prob.append(tempt)
result=[]
for i in range(len(prob)):
    result.append([test_nums[i],prob[i]])
column=['ID','Prediction']  
test=pd.DataFrame(columns=column,data=result)
test.to_csv('/home/kesci/work/test_submission.csv')